In [1]:
%load_ext autoreload
%autoreload

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
import re
from keras.layers import Input, Dense
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, TensorBoard
from keras import metrics
import json
from helper_functions import import_data
from sklearn.metrics import confusion_matrix
from os import listdir



Using TensorFlow backend.


In [28]:
len(listdir('jsons'))

1000

In [7]:
test_json_path = 'jsons/part-00001-3a3b13a1-b857-4d1c-965f-ce7bf860f3bc-c000.json'
dense_matrix, y_vals = import_data(test_json_path)
X_test = dense_matrix[[0:1000, :]]
X_train = dense_matrix[1000:, :]

y_test = y_vals[0:1000]
y_train = y_vals[1000:]
X_test.shape

(1000, 15133)

In [24]:
dense_matrix[y_vals == 0].shape

(536, 15133)

In [9]:
def autoencoder_model(X_train):
    '''
    defines autoencoder model
    input: X_train (2D np array)
    output: autoencoder (compiled autoencoder model)
    '''
    # this is our input placeholder
    input_img = Input(shape=(X_train.shape[1],))

    # first encoding layer
    encoded1 = Dense(units = 1000, activation = 'relu', name='layer1_256')(input_img)

    # second encoding layer
    # note that each layer is multiplied by the layer before
    encoded2 = Dense(units = 200, activation='relu', name='layer2_64')(encoded1)

    # first decoding layer
    decoded1 = Dense(units = 1000, activation='relu', name='layer3_256')(encoded2)

    # second decoding layer - this produces the output
    decoded2 = Dense(units = X_train.shape[1], activation='sigmoid', name='layer4_output')(decoded1)

    # this model maps an input to its reconstruction
    autoencoder = Model(input_img, decoded2)

    # compile model
    autoencoder.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['mse'])

    return autoencoder

In [35]:
dir_list = listdir('jsons')
np.random.randint(0, len(dir_list))


1000

In [4]:
samples_per_epoch = 100

In [12]:
def batch_generator(file_dir, batch_size, autoencoder_layer=True):
    #creates list of filename strings from jsons directory
    dir_list = listdir('jsons')
    
    #Determines number of batches to produce
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    file_counter = 0
    
    #Creates random shuffle index
    file_shuffle_index = np.arange(len(dir_list))
    np.random.shuffle(file_shuffle_index)
    
    #Uses first random shuffle index to select a file
    X, y = import_data('jsons/' + dir_list[file_shuffle_index[file_counter]])
    
    #Creates shuffle index arrays to randomize the data rows as well as the files used
    data_shuffle_index = np.arange(np.shape(X)[0])
    np.random.shuffle(data_shuffle_index)
    
    while 1:
        index_batch = data_shuffle_index[batch_size*counter:batch_size*(counter+1)]
        
        #Saves the smallest count of good/bad reviews 
        smallest_good_or_bad_count = min([y[y == 0].shape[0], y[y == 0].shape[0]])
        
        #Creates an array with a 50/50 split of good/bad reviews
        X_batch_good = X[y == 1][0:smallest_good_or_bad_count, :]
        X_batch_bad = X[y == 0][0:smallest_good_or_bad_count, :]
        y_batch_good = y[y == 1][0:smallest_good_or_bad_count]
        y_batch_bad = y[y == 0][0:smallest_good_or_bad_count]
        
        #Stacks the splits
        X_batch = np.vstack((X_batch_good, X_batch_bad))
        y_batch = np.vstack((y_batch_good.reshape(-1, 1), y_batch_bad.reshape(-1,1)))
        
        #Randomizes the reviews so there isn't a block of good/bad reviews
        batch_shuffle_index = np.arange(X_batch.shape[0])
        np.random.shuffle(batch_shuffle_index)
        X_batch = X_batch[batch_shuffle_index, :]
        print(y_batch.shape)
        y_batch = y_batch[batch_shuffle_index]
        
        counter += 1
        
        if autoencoder_layer:
            yield X_batch, X_batch
        else:
            yield X_batch, y_batch
        
        
        if (counter > number_of_batches):
            counter=0
            file_count +=1
            
            if file_count == len(dir_list):
                np.random.shuffle(file_shuffle_index)
                file_count = 0
                
            np.random.shuffle(shuffle_index)
            X, y = import_data('jsons/' + dir_list[file_shuffle_index[file_counter]])
            

In [13]:
for X_, y_ in batch_generator('jsons', 100, False):
    X_test = X_
    y = y_
    break

(1124, 1)


In [16]:
y[y==0].shape

(562,)

In [70]:
smallest_good_or_bad_count = min([y[y == 0].shape[0], y[y == 0].shape[0]])
smallest_good_or_bad_count

551

In [67]:
X_test

array([[ 3.,  3.,  0., ...,  0.,  0.,  0.],
       [18.,  7.,  6., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 3.,  1.,  2., ...,  0.,  0.,  0.],
       [ 3.,  0.,  2., ...,  0.,  0.,  0.]])

In [40]:
X, y = import_data('jsons/' + listdir('jsons')[0])

In [ ]:
X, y = import_data(listdir('jsons')[0])

In [44]:
min([y[y == 0].shape[0], y[y == 0].shape[0]])

551

In [11]:
autoencoder_model_created = False
model_path = 'models/basic_autoencoder1.h5'

if not autoencoder_model_created:
    model = autoencoder_model(X_train)

    batch_size = 1000
    nb_epoch = 10
    samples_per_epoch = 15

    # instantiate callbacks
    tensorboard = TensorBoard(log_dir='./autoencoder_logs', histogram_freq=2, batch_size=batch_size, write_graph=True, write_grads=True, write_images=True)
    earlystopping = EarlyStopping(monitor='val_loss', patience=2)

    # try different number of epochs - 10 gives good performanace 
    """model.fit(X_train, X_train, epochs=10, batch_size=batch_size, verbose=1,
              validation_split=0.1, callbacks = [earlystopping, tensorboard])""" # cross val to estimate test error


    model.fit_generator(generator=batch_generator('jsons', batch_size),
                        epochs=nb_epoch,
                        steps_per_epoch=samples_per_epoch)


    scores = model.evaluate(X_test, X_test)
    print('Test mse = {}'.format(scores[0]))

    X_test_decoded = model.predict(X_test)
    
    model.save(model_path)

else:
    model = load_model(model_path)
    scores = model.evaluate(X_test, X_test)
    print('Test mse = {}'.format(scores[0]))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
15/15 [==============================] - 4s 237ms/step - loss: 0.1446 - mean_squared_error: 0.1446
Epoch 2/10
15/15 [==============================] - 2s 147ms/step - loss: 0.0373 - mean_squared_error: 0.0373
Epoch 3/10
15/15 [==============================] - 2s 145ms/step - loss: 0.0357 - mean_squared_error: 0.0357
Epoch 4/10
15/15 [==============================] - 2s 146ms/step - loss: 0.0363 - mean_squared_error: 0.0363
Epoch 5/10
15/15 [==============================] - 2s 146ms/step - loss: 0.0372 - mean_squared_error: 0.0372
Epoch 6/10
15/15 [==============================] - 2s 149ms/step - loss: 0.0371 - mean_squared_error: 0.0371
Epoch 7/10
15/15 [==============================] - 2s 146ms/step - loss: 0.0355 - mean_squared_error: 0.0355
Epoch 8/10
15/15 [==============================] - 2s 145ms/step - loss: 0.0369 - mean_squared_error: 0.0369


In [12]:
for i in range(4):
    print(model.layers[i].name)

input_1
layer1_256
layer2_64
layer3_256


In [13]:
for i in range(4):
    print(model.layers[i].name)
    model.layers[i].trainable = False

input_1
layer1_256
layer2_64
layer3_256


In [21]:
ll = model.layers[3].output
ll = Dense(units = 64, activation='relu', name='layer4_256')(ll)
ll = Dense(1,activation="hard_sigmoid", name='positive_classification')(ll)
new_model = Model(inputs=model.input, outputs=ll)

new_model.compile(optimizer = 'adam', loss = 'mse', metrics=[metrics.categorical_accuracy])

In [22]:
batch_size = 500
nb_epoch = 30
samples_per_epoch = 15
model_path = 'models/basic_autoencoder1.h5'

# instantiate callbacks
tensorboard = TensorBoard(log_dir='./autoencoder_logs', histogram_freq=2, batch_size=batch_size, write_graph=True, write_grads=True, write_images=True)
earlystopping = EarlyStopping(monitor='val_loss', patience=2)

# try different number of epochs - 10 gives good performanace 
"""model.fit(X_train, X_train, epochs=10, batch_size=batch_size, verbose=1,
          validation_split=0.1, callbacks = [earlystopping, tensorboard])""" # cross val to estimate test error


new_model.fit_generator(generator=batch_generator('jsons', batch_size, autoencoder_layer=False),
                    epochs=nb_epoch,
                    steps_per_epoch=samples_per_epoch)


scores = new_model.evaluate(X_test, y_test)
print('Test accuracy = {}'.format(scores[1]))

X_test_decoded = new_model.predict(X_test)

new_model.save(model_path)

Epoch 1/30


/home/tyler/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if sys.path[0] == '':


15/15 [==============================] - 1s 39ms/step - loss: 0.1611 - categorical_accuracy: 1.0000
Epoch 2/30
15/15 [==============================] - 0s 23ms/step - loss: 0.1161 - categorical_accuracy: 1.0000
Epoch 3/30
15/15 [==============================] - 0s 23ms/step - loss: 0.1167 - categorical_accuracy: 1.0000
Epoch 4/30
15/15 [==============================] - 0s 23ms/step - loss: 0.1095 - categorical_accuracy: 1.0000
Epoch 5/30
15/15 [==============================] - 0s 23ms/step - loss: 0.1145 - categorical_accuracy: 1.0000
Epoch 6/30
15/15 [==============================] - 0s 23ms/step - loss: 0.1161 - categorical_accuracy: 1.0000
Epoch 7/30
15/15 [==============================] - 0s 23ms/step - loss: 0.1101 - categorical_accuracy: 1.0000
Epoch 8/30
15/15 [==============================] - 0s 23ms/step - loss: 0.1160 - categorical_accuracy: 1.0000
Epoch 9/30
15/15 [==============================] - 0s 23ms/step - loss: 0.1163 - categorical_accuracy: 1.0000
Epoch 10/30


In [18]:
X_test_decoded

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],

In [20]:
confusion_matrix(y_test, X_test_decoded.astype(int))

array([[  0, 116],
       [  0, 884]])

In [29]:
y_test

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

In [32]:
X_test_decoded.astype(int)

array([[0, 0],
       [0, 0],
       [0, 0],
       ...,
       [0, 0],
       [0, 0],
       [0, 0]])